<a href="https://colab.research.google.com/github/srinivasdasu24/__DataScience/blob/main/HR_Analytics/InClass_Working_Code_6_Decision_Tree_Base_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code 6
- Preprocessing Improvement
- Decision Tree

## 1/ Import Libraries

In [1]:
#CodeSection1
from google.colab import drive
drive.mount('/mntDrive') 

Mounted at /mntDrive


In [2]:
#CodeSection2
import pandas as pd
import numpy as np

## 2/ Import Data

In [3]:
#CodeSection3
train = pd.read_csv("/mntDrive/MyDrive/__DataScience/3 HR Analytics/input/train.csv")
test = pd.read_csv("/mntDrive/MyDrive/__DataScience/3 HR Analytics/input/test.csv")

## 3/ Combine both Datasets (train, test)

In [4]:
#CodeSection4
all_data = [train, test]

### Check Combined Data 
- Missing Value
- Data Types

In [5]:
#CodeSection5
# Combine both to check Missing Value
pd.concat([train, test], axis = 0).isnull().sum()

enrollee_id                   0
city                          0
city_development_index        0
gender                     7486
relevent_experience           0
enrolled_university         621
education_level             852
major_discipline           5231
experience                  103
company_size               8830
company_type               9369
last_new_job                671
training_hours                0
target                    15021
dtype: int64

## 4/ Preprocessing Improvement

### 4.1/ Identify Numerical and Categorical Features

In [6]:
#CodeSection6
# Identify all Numerical and Categorical features
numerical_features = ['training_hours']
categorical_features = ['gender', 'enrolled_university', 'major_discipline', 'education_level', 'company_size', 'company_type', 'experience', 'last_new_job']

### 4.2/ Outlier Strategy

In [7]:
#CodeSection7

# Write a loop to do the same
for num_var in numerical_features:
  Q1 = train[num_var].quantile(0.25)
  Q3 = train[num_var].quantile(0.75)

  IQR = Q3-Q1

  Lower_Whisker = Q1 - 1.5*IQR
  Upper_Whisker = Q3 + 1.5*IQR

  train[num_var] = train[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)
  test[num_var] = test[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)

### 4.3/ Import Libraries
- Missing Value Imputation - SimpleImputer
- Preprocessing - StandardScaler, OrdinalEncoder
- Pipeline - make_pipeline, make_column_transformer
- Model - Decision Tree

In [8]:
#CodeSection8

# Import SimpleImputer
from sklearn.impute import SimpleImputer

# Import StandardScaler
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

# Make and Compose Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

# Import Decision tree
from sklearn.tree import DecisionTreeClassifier

### 4.4/ Build Pipeline

In [9]:
#CodeSection9
# Create Preprocessor Pipeline
preprocessor = make_column_transformer(
    (make_pipeline(SimpleImputer(strategy='median'),
      StandardScaler()), numerical_features),

    (make_pipeline(SimpleImputer(strategy='most_frequent'),
                  OrdinalEncoder()), categorical_features)
        
)

### 4.5/ Divide Data into X and y

In [10]:
#CodeSection10
X = train.drop(['enrollee_id','target'], axis =  1)
y = train['target']

## 5/ Build Model and Fit

In [11]:
#CodeSection11
# Create Model Pipeline and Initiate Model
model = make_pipeline(preprocessor, DecisionTreeClassifier(max_depth=5))

In [12]:
#CodeSection12 
# Fit Model
model.fit(X,y)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

## 6/ Check Accuracy of Model on Train Data

In [13]:
#CodeSection13
# Predict on Train Data
y_pred = model.predict(X)

### We can use a Accuracy Function from Metrics
- Check Train Accuracy

In [14]:
#CodeSection14
# Import metrics library
from sklearn.metrics import accuracy_score

In [15]:
#CodeSection15
# get Actual "y" variables (use "y_true" as variable)
y_true = train['target']

In [16]:
#CodeSection16
# Print Train Accuracy
print(f" Train Accuracy : {accuracy_score(y_true, y_pred):0.1%}")

 Train Accuracy : 86.8%


## 7/ Predict and Submission

### Predict on "Test Data"

In [17]:
#CodeSection17
# Get all the X Variables from the Test Dataset
X_test = test.drop(['enrollee_id'], axis =  1)

In [18]:
#CodeSection18
# Predict on X_test Data ("X_test_prep")
X_test_prep = model.predict(X_test)

### Create Submission File

In [19]:
#CodeSection19
submission = pd.DataFrame({
    'enrollee_id' : test['enrollee_id'],
    'target' : X_test_prep
})

### Export Submission File

In [20]:
#CodeSection20
submission.to_csv('/mntDrive/MyDrive/__DataScience/3 HR Analytics/output/O6_DT_Pipeline.csv', index = False)

In [21]:
# LB Accuracy : 0.5002